## Analizador Lexico
Un analizador lexico transforma una entrada en tokens

In [37]:
entrada = "Si si es de dia entonces mañana no es de noche"

In [38]:
tokens = entrada.lower().split()
tokens

['si',
 'si',
 'es',
 'de',
 'dia',
 'entonces',
 'mañana',
 'no',
 'es',
 'de',
 'noche']

In [39]:
from enum import Enum

class TipoToken(Enum):
    O = 1
    Y = 2
    S = 3
    N = 4
    E = 5
    P = 6

In [40]:

lexemas = []
for token in tokens:
    if token == 'o':
        lexemas.append({"valor" : token , "tipo": TipoToken.O })
    elif token == 'y':
        lexemas.append({"valor" : token , "tipo": TipoToken.Y})
    elif token == 'si':
        lexemas.append({"valor" : token , "tipo": TipoToken.S})
    elif token == 'entonces':
        lexemas.append({"valor" : token , "tipo": TipoToken.E})
    elif token == 'no':
        lexemas.append({"valor" : token , "tipo": TipoToken.N})
    else :
        lexemas.append({"valor" :token , "tipo": TipoToken.P })
        
lexemas

[{'valor': 'si', 'tipo': <TipoToken.S: 3>},
 {'valor': 'si', 'tipo': <TipoToken.S: 3>},
 {'valor': 'es', 'tipo': <TipoToken.P: 6>},
 {'valor': 'de', 'tipo': <TipoToken.P: 6>},
 {'valor': 'dia', 'tipo': <TipoToken.P: 6>},
 {'valor': 'entonces', 'tipo': <TipoToken.E: 5>},
 {'valor': 'mañana', 'tipo': <TipoToken.P: 6>},
 {'valor': 'no', 'tipo': <TipoToken.N: 4>},
 {'valor': 'es', 'tipo': <TipoToken.P: 6>},
 {'valor': 'de', 'tipo': <TipoToken.P: 6>},
 {'valor': 'noche', 'tipo': <TipoToken.P: 6>}]

## Analizador Sintactico

In [41]:
class EstadosAnalizador(Enum):
    INICIO = 1
    PRECEDENTE = 2
    ERROR1 = 3
    NEGACION = 4
    CONSECUENTE = 5
    ATOMO = 6
    DISYUNCION = 7
    CONJUNCION = 8
    ERROR2 = 9
    FIN = 10

In [42]:
def analizador_sintactico(lexemas):
    EstadoActual = EstadosAnalizador.INICIO

    for lexema in lexemas:
        if EstadoActual == EstadosAnalizador.INICIO:
            if lexema['valor'] == "si":
                EstadoActual = EstadosAnalizador.PRECEDENTE
            else:
                EstadoActual = EstadosAnalizador.ERROR1
        elif EstadoActual == EstadosAnalizador.PRECEDENTE:
            if lexema['valor'] == "no":
                EstadoActual = EstadosAnalizador.NEGACION
            elif lexema['valor'] == "es":
                EstadoActual = EstadosAnalizador.CONSECUENTE
            else:
                EstadoActual = EstadosAnalizador.ERROR2
        elif EstadoActual == EstadosAnalizador.NEGACION:
            if lexema['valor'] == "de":
                EstadoActual = EstadosAnalizador.ATOMO
            else:
                EstadoActual = EstadosAnalizador.ERROR2
        elif EstadoActual == EstadosAnalizador.CONSECUENTE:
            if lexema['valor'] == "de":
                EstadoActual = EstadosAnalizador.ATOMO
            else:
                EstadoActual = EstadosAnalizador.ERROR2
        elif EstadoActual == EstadosAnalizador.ATOMO:
            if lexema['valor'] == "entonces":
                EstadoActual = EstadosAnalizador.FIN
            elif lexema['valor'] == "y":
                EstadoActual = EstadosAnalizador.CONJUNCION
            elif lexema['valor'] == "o":
                EstadoActual = EstadosAnalizador.DISYUNCION
        elif EstadoActual == EstadosAnalizador.CONJUNCION:
            if lexema['valor'] == "de":
                EstadoActual = EstadosAnalizador.ATOMO
            else:
                EstadoActual = EstadosAnalizador.ERROR2
        elif EstadoActual == EstadosAnalizador.DISYUNCION:
            if lexema['valor'] == "de":
                EstadoActual = EstadosAnalizador.ATOMO
            else:
                EstadoActual = EstadosAnalizador.ERROR2
        elif EstadoActual == EstadosAnalizador.ERROR1:
            break
        elif EstadoActual == EstadosAnalizador.ERROR2:
            break
        elif EstadoActual == EstadosAnalizador.FIN:
            break

    if EstadoActual == EstadosAnalizador.FIN:
        print("La frase fue bien escrita.")
    else:
        print("La frase contiene un error sintáctico.")

    return EstadoActual

# Ejecución del analizador sintáctico
estado_final = analizador_sintactico(lexemas)

La frase contiene un error sintáctico.


In [43]:
print("Estado final:", estado_final)
    

Estado final: EstadosAnalizador.ERROR2


#Arboles


In [44]:
class Nodo: 
    def __init__(self, regla):
        self.regla = regla
        self.izquierda = None
        self.derecha = None


nodo1 = Nodo("El sol es amarillo y la luna es negra.")
nodo2 = Nodo("Si el sol es amarillo, entonces la luna no puede ser amarilla.")
nodo3 = Nodo("La luna es blanca entonces, el sol no puede ser blanco.")
nodo4 = Nodo("Cuando el sol es blanco, la luna no puede ser blanca.")
nodo5 = Nodo("Si la luna es blanca, el sol no puede ser blanco.")
nodo6 = Nodo("Si el sol es negro o la luna es blanca.")
nodo7 = Nodo("la luna es negra o el sol es negro.")
nodo8 = Nodo("Si la luna no es blanca, entonces el sol es blanco.")
nodo9 = Nodo("El sol o la luna no pueden ser ambos negros.")
nodo10 = Nodo("Si el sol y la luna son blancos no es de día.")

nodo1.izquierda = nodo3
nodo1.derecha = nodo2
nodo3.izquierda = nodo4
nodo3.derecha = nodo5
nodo5.izquierda = nodo6
nodo5.derecha = nodo7
nodo7.izquierda = nodo8
nodo7.derecha = nodo9
nodo8.izquierda = nodo10

def recorrer_arbol(nodo, si_phrases, no_si_phrases):
    if nodo is not None:
        recorrer_arbol(nodo.izquierda, si_phrases, no_si_phrases)
        if nodo.regla.startswith("Si"):
            si_phrases.append(nodo.regla)
        else:
            no_si_phrases.append(nodo.regla)
        recorrer_arbol(nodo.derecha, si_phrases, no_si_phrases)


si_phrases = []
no_si_phrases = []
recorrer_arbol(nodo1, si_phrases, no_si_phrases)

print("Aquí están las frases que empiezan con 'Si':")
for frase in si_phrases:
    print(frase)


print("\nAquí están las frases que no empiezan con 'Si':")
for frase in no_si_phrases:
    print(frase)

tabla = []
for i, frase in enumerate(si_phrases + no_si_phrases, start=1):
    tabla.append((frase, i))

Aquí están las frases que empiezan con 'Si':
Si el sol es negro o la luna es blanca.
Si la luna es blanca, el sol no puede ser blanco.
Si el sol y la luna son blancos no es de día.
Si la luna no es blanca, entonces el sol es blanco.
Si el sol es amarillo, entonces la luna no puede ser amarilla.

Aquí están las frases que no empiezan con 'Si':
Cuando el sol es blanco, la luna no puede ser blanca.
La luna es blanca entonces, el sol no puede ser blanco.
la luna es negra o el sol es negro.
El sol o la luna no pueden ser ambos negros.
El sol es amarillo y la luna es negra.
